### Advanced Machine Learning

# Home Exercise #1

Assignment Due: 21/03/21 23:59

Jonathan Schler


In [ ]:
student1Name="Alexander Kruglyak" # student1 name here
student1ID="323637736" # student1 ID here


student2Name="Sofya Zubtsovsky" # student2 name here
student2ID="337839112" # student2 ID here


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, LinearRegression, Lasso, RidgeCV, LassoCV, ElasticNet
from sklearn.impute import KNNImputer,SimpleImputer
import seaborn as sns

%matplotlib inline

# add more imports as needed

## Task 1 Regression on Ames Housing Dataset (60 Points)

You can load the Ames housing dataset from <br>
http://www.amstat.org/publications/jse/v19n3/decock/AmesHousing.xls <br>
You can find a description of the variables here: <br>
http://jse.amstat.org/v19n3/decock/DataDocumentation.txt <br>
Take note that for categorical variables, NA here does not mean a missing value, but should be treated as a separate category.

In [ ]:
def read_data():
    return pd.read_excel("AmesHousing.xls")
df = read_data()
df

In [ ]:
NA_FEATURES = [
"Mas Vnr Type",
"Bsmt Qual",
"Bsmt Cond",
"Bsmt Exposure",
"BsmtFin Type 1",
"BsmtFin Type 2",
"Electrical",
"Garage Type",
"Garage Finish",
"Garage Qual",
"Garage Cond"
]
# "Misc Feature", "Fence", "Pool QC", "Alley" - because only ~200 rows contains data - less than 10%
COLUMNS_TO_DROP = [
    "SalePrice", 
    "PID", 
    "Order", 
    "Misc Feature", 
    "Fence", 
    "Pool QC", 
    "Alley", 
    "Fireplace Qu"
] 
# + NA_FEATURES


In [ ]:
def drop_columns(df, columns):
    return df.copy().drop(columns, axis=1)


dropped_df = drop_columns(df, COLUMNS_TO_DROP)
continuous_features = dropped_df.select_dtypes(exclude=['object'])
categorical_features = dropped_df.select_dtypes(include=['object'])

### Question 1.1
Visualize the univariate distribution of each continuous attribute, and the distribution of the target. Make sure you skip the string\categorial columns (you can do it using call to `select_dtypes(exclude=['object'])`).

Do you notice anything? Is there something that might require special treatment?

In [ ]:
# todo: we need to know how to take the whole data
y = df.SalePrice
y_limit = y.max()
y.max()

In [ ]:
features_description = {
    "MS SubClass": "type of dwelling involved in the sale",
    "Lot Frontage": "Linear feet of street connected to property",
    "Lot Area": "Lot size in square feet",
    "Overall Qual": "Rates the overall material and finish of the house",
    "Overall Cond": "Rates the overall condition of the house",
    "Year Built": "Original construction date",
    "Year Remod/Add": "Remodel date (same as construction date if no remodeling or additions)",
    "Mas Vnr Area": "Masonry veneer area in square feet",
    "BsmtFin SF 1": "Type 1 finished square feet",
    "BsmtFin SF 2": "Type 2 finished square feet",
    "Bsmt Unf SF": "Unfinished square feet of basement area",
    "Total Bsmt SF": "Total square feet of basement area",
    "1st Flr SF": "First Floor square feet",
    "2nd Flr SF": "Second floor square feet",
    "Low Qual Fin SF": "Low quality finished square feet (all floors)",
    "Gr Liv Area": "Above grade (ground) living area square feet",
    "Bsmt Full Bath": "Basement full bathrooms",
    "Bsmt Half Bath": "Basement half bathrooms",
    "Bsmt Full Bath": "Basement full bathrooms",
    "Full Bath": "Full bathrooms above grade",
    "Half Bath": "Half baths above grade",
    "Fireplaces": "Number of fireplaces",
    "Misc Val": "Value of miscellaneous feature",
    "3Ssn Porch": "Three season porch area in square feet"
}

title_font = {
        'color':  'darkblue',
        'weight': 'normal',
        'size': 20,
        }
labels_font = {
        'color':  'darkred',
        'weight': 'normal',
        'size': 16,
        }

In [ ]:
fig, axes = plt.subplots(12, 3, figsize=(20, 70))

for i, ax in enumerate(axes.ravel()):
    feature_name = continuous_features.columns[i]
    feature_human_readable_name = features_description.get(feature_name, "===NONE===")
    y = continuous_features[feature_name].value_counts().sort_index()
    ax.set_title("{} / \n {}".format(feature_name, feature_human_readable_name), fontdict=title_font)
    ax.set_ylabel("Counts", fontdict=labels_font)
    ax.plot(y)
    ax.grid()
plt.tight_layout()


### The plot allows to define the following:
most of the feautures must be scaled and normalized:
<ul>
<li>lot frontage, lot area, total basement SF, garage year build and many other features aren't distributed around the median (there is a skew of the data)</li>
<li>second floor, low quality SF, area and other features have a number of extreme values, and should be presented using log axis</li>
<li>features have different scale, distribution and standard deviation</li></ul>

### Question 1.2
Visualize the dependency of the target on each continuous feature (2d scatter plot).

In [ ]:
fig, axes = plt.subplots(12, 3, figsize=(20, 70))
y = df.SalePrice

for i, ax in enumerate(axes.ravel()):
    X_col = continuous_features.iloc[:, i]
    ax.set_ylim(0, y_limit)
    if X_col.nunique() > 10:
        ax.set_xlim(X_col.quantile(q=.0001), X_col.quantile(q=.9999))
        
    ax.scatter(X_col, y, alpha=.5)

    feature_name = continuous_features.columns[i]
    feature_human_readable_name = features_description.get(feature_name, "===NONE===")
    ax.set_title("{} / \n {}".format(feature_name, feature_human_readable_name), fontdict=title_font)
    ax.set_ylabel("Price ($)", fontdict=labels_font)
    ax.grid()
    
plt.tight_layout()

### Question 1.3
Split data in training and test set. Do not use the test-set unless for a final evaluation in 1.6.
For each categorical variable, cross-validate a Linear Regression model using just this variable
(one-hot-encoded). Visualize the relationship of the categorical variables that provide the best
R^2 value with the target.

In [ ]:
#plot the price to see how it's spreaded across the indexes
df.plot.line(y='SalePrice')


In [ ]:
X = dropped_df
#get dummies
X_dummies = pd.get_dummies(X)
#method 1 - split data in the ordinary way
# what about stratify???
X_train, X_test, y_train, y_test = train_test_split(X_dummies,y,random_state=0)

#method 2 - split with stratification
nof_bins = 50 #can be adjusted
bins = np.linspace(np.min(y), len(y), nof_bins)
# split continious SalePrice  into bins to make it 'categorical'
y_binned = np.digitize(y, bins)
X_stratified_train, X_stratified_test, y_stratified_train, y_stratified_test = train_test_split(X_dummies, y, stratify = y_binned,random_state=0)
print ('median without stratify: train %f test %f\nmedian with stratify: train %f, test %f' % (np.median(y_train), np.median(y_test),np.median(y_stratified_train), np.median(y_stratified_test)))

#TBD check if the order between split and get_dummies is correct

In [ ]:
# todo: refactor to function
for f in categorical_features.columns:
    if categorical_features[f].isna().sum() /len(categorical_features[f]) > 0.3:
        print(f)

In [ ]:
categorical_features.isna().sum()

In [ ]:
# categorical_features.plot(kind='box')
#TBD add plot in a loop (doesn't work as expected)
df.boxplot('SalePrice','Sale Condition',rot = 30,figsize=(5,6))

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

# todo: consider better approach: with imputation
# categorical_features = categorical_features.dropna()
X_train, X_test, y_train, y_test = train_test_split(categorical_features, y, random_state=0)

r2_scores = np.empty((0,2))
for column in X_train:
        two_d = X_train[column].values.reshape(-1,1)        
        array_hot_encoded = OneHotEncoder(handle_unknown='ignore').fit_transform(two_d).toarray()
        data_hot_encoded_df = pd.DataFrame(array_hot_encoded, index=X_train.index)
        score = np.mean(cross_val_score(LinearRegression(), data_hot_encoded_df, y_train, cv=10))
        r2_scores = np.concatenate((r2_scores, [[column, score]]))
# r2_scores
r2_scores = r2_scores[r2_scores[:,1].argsort()]
r2_scores

In [ ]:
[feature_name, score] = r2_scores[-1]
feature_name
data = categorical_features[feature_name]
data
# data.value_counts().plot(kind='bar')
plt.scatter(data, y)
plt.show()
# ax = data.plot.bar(x=data, y=y_train, rot=0)

In [ ]:
COLUMNS_TO_DROP_COPY = COLUMNS_TO_DROP.copy()
COLUMNS_TO_DROP_COPY.remove("SalePrice")
data = drop_columns(
    read_data(),
    COLUMNS_TO_DROP_COPY
)

# data["SalePrice"]
[feature_name, score] = r2_scores[-1]
data.loc[dropped_df['Exter Qual'] == 'Gd']
categories_within_feature = data[feature_name].unique()
categories_within_feature

prices_by_category = []
for category in categories_within_feature:
    # this will not work, we need to create in every array as much as categories variables
    price_by_category = data[data[feature_name] == category]["SalePrice"].values
#     print(price_by_category)
    prices_by_category.append(price_by_category)
    
prices_by_category
#     print(prices_by_category)
# data = categorical_features[feature_name]
price = np.random.rand(10, 4)
price
# categories_within_feature = data.unique()

# df = pd.DataFrame(prices_by_category, columns=categories_within_feature)
# df.plot.box()

In [ ]:
# categorical_features.index.tolist()
fig, axes = plt.subplots(9, 3, figsize=(20, 70))
y = df['SalePrice']
print (len(y))
for i, ax in enumerate(axes.ravel()):
    f = categorical_features.columns.values[i]
    print(f)
    data = categorical_features[f]
    # ax.scatter(data, y, alpha=.5)
    categorical_features.boxplot(y,by=f,ax=ax)
    # ax.boxplot(y,data)
    ax.set_title("{}".format(f), fontdict=title_font)
    ax.set_ylabel("Price ($)", fontdict=labels_font)
    ax.grid()
    
plt.tight_layout()

### Question 1.4
Use ColumnTransformer and pipeline to encode categorical variables. Evaluate Linear
Regression (OLS), Ridge, Lasso and ElasticNet using cross-validation with the default
parameters. Does scaling the data (within the pipeline) with StandardScaler help?

In [ ]:
def impute_numeric(df):
    return pd.DataFrame(KNNImputer().fit_transform(df),columns = df.columns,index=df.index)

numeric = dropped_df.select_dtypes(exclude=['object'])
print  ("before\n",numeric.isna().sum())
numeric = impute_numeric(numeric)
print ("after\n",numeric.isna().sum())
numeric

In [ ]:
def impute_categorical(df):
    return df.fillna("missing")
categorical = dropped_df.select_dtypes(include=['object'])
categorical = impute_categorical(categorical)
categorical.isna().sum()
categorical

## Preprocess data

In [117]:
numeric = impute_numeric(dropped_df.select_dtypes(exclude=['object']))
categorical = impute_categorical(dropped_df.select_dtypes(include=['object']))

#TBD consider how to fill columns of inplace: fill & KNNImputer does not fill in place when we pass a slice of dataframe
# or think how to concatenate categorical/numeric columns after imputing

# X_train, X_test, y_train, y_test = train_test_split(df_imputed, y, random_state=0)



# preprocess_scaled = make_column_transformer(
#     (StandardScaler(), ~categorical),
#     (OneHotEncoder(), categorical)
# )

# preprocess_not_scaled = make_column_transformer(
#     (OneHotEncoder(), categorical)
# )

# # disregard warning
# rergressors = [LinearRegression(),Ridge(),Lasso(),ElasticNet()] 
# for regr in rergressors:
#     non_caled_pipe = make_pipeline(preprocess_not_scaled,regr)
#     scaled_pipe = make_pipeline(preprocess_scaled,regr)
#     scores[regr] = (np.mean(cross_val_score(scaled_pipe, X_train, y_train)),np.mean(cross_val_score(non_caled_pipe, X_train, y_train)))
#     break

,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,...,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold
0,20.0,141.0,31770.0,6.0,5.0,1960.0,1960.0,112.0,639.0,0.0,...,528.0,210.0,62.0,0.0,0.0,0.0,0.0,0.0,5.0,2010.0
1,20.0,80.0,11622.0,5.0,6.0,1961.0,1961.0,0.0,468.0,144.0,...,730.0,140.0,0.0,0.0,0.0,120.0,0.0,0.0,6.0,2010.0
2,20.0,81.0,14267.0,6.0,6.0,1958.0,1958.0,108.0,923.0,0.0,...,312.0,393.0,36.0,0.0,0.0,0.0,0.0,12500.0,6.0,2010.0
3,20.0,93.0,11160.0,7.0,5.0,1968.0,1968.0,0.0,1065.0,0.0,...,522.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2010.0
4,60.0,74.0,13830.0,5.0,5.0,1997.0,1998.0,0.0,791.0,0.0,...,482.0,212.0,34.0,0.0,0.0,0.0,0.0,0.0,3.0,2010.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,80.0,37.0,7937.0,6.0,6.0,1984.0,1984.0,0.0,819.0,0.0,...,588.0,120.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2006.0
2926,20.0,79.0,8885.0,5.0,5.0,1983.0,1983.0,0.0,301.0,324.0,...,484.0,164.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2006.0
2927,85.0,62.0,10441.0,5.0,5.0,1992.0,1992.0,0.0,337.0,0.0,...,0.0,80.0,32.0,0.0,0.0,0.0,0.0,700.0,7.0,2006.0
2928,20.0,77.0,10010.0,5.0,5.0,1974.0,1975.0,0.0,1071.0,123.0,...,418.0,240.0,38.0,0.0,0.0,0.0,0.0,0.0,4.0,2006.0


In [ ]:
#take only categorical values from train set
categorical = X_train.dtypes == object

scores = {}
preprocess = make_column_transformer(
    (StandardScaler(), ~categorical),
    (OneHotEncoder(), categorical))
#disregard warning
rergressors = [LinearRegression(),Ridge(),Lasso(),ElasticNet()] 
for regr in rergressors:
    scaled_pipe = make_pipeline(KNNImputer(),preprocess,regr) #important to impute NaN values to avoid errors on regression
    non_caled_pipe = make_pipeline(KNNImputer(),regr)
    scores[regr] = (np.mean(cross_val_score(scaled_pipe, X_train, y_train)),np.mean(cross_val_score(non_caled_pipe, X_train, y_train)))

In [ ]:
df = pd.DataFrame(data=scores,index=['scaled','non-scaled'],dtype=float)    
print(df)

### Question 1.5
Tune the parameters of the models using GridSearchCV. Do the results improve?
Visualize the dependence of the validation score on the parameters for Ridge, Lasso and
ElasticNet.

In [ ]:
categorical = X_train.dtypes == object
preprocess = make_column_transformer(
    (StandardScaler(), ~categorical),
    (OneHotEncoder(), categorical))

In [ ]:
pipe = Pipeline([('imputer', KNNImputer()),
                 ('scaler', preprocess),
                 ('regressor', Ridge())])

In [ ]:
def plot_alpha_score(gridcv_results):
    results = pd.DataFrame(gridcv_results)
    results.plot('param_regressor__alpha', 'mean_train_score')
    results.plot('param_regressor__alpha', 'mean_test_score', ax=plt.gca())
    plt.fill_between(results.param_regressor__alpha.astype(np.float),
                     results['mean_train_score'] + results['std_train_score'],
                     results['mean_train_score'] - results['std_train_score'], alpha=0.2)
    plt.fill_between(results.param_regressor__alpha.astype(np.float),
                     results['mean_test_score'] + results['std_test_score'],
                     results['mean_test_score'] - results['std_test_score'], alpha=0.2)
    plt.legend()
    plt.xscale("log")

In [ ]:
def plot_alpha_l1_score(gridcv_results):
    res = pd.pivot_table(pd.DataFrame(gridcv_results),
    values='mean_test_score', index='param_regressor__alpha', columns='param_regressor__l1_ratio')
    res = res.set_index(res.index.values.round(4))
    sns.heatmap(res, annot=True, fmt=".3g", vmin=0.6)

In [ ]:
#TBD decide if we need to use array of regressor instead of validate one by one
# multiple_param_grid = [{'imputer': [KNNImputer()],
#             'regressor': [Ridge()],
#             'regressor__alpha': np.logspace(-3, 3, 13),
#             'scaler': [preprocess]},
#             {'imputer': [KNNImputer()],
#              'regressor': [Lasso()],
#             'regressor__alpha': np.logspace(-3, 0, 13),
#             'scaler': [preprocess]},
#             {'imputer': [KNNImputer()],
#             'regressor': [ElasticNet()],
#             'regressor__alpha': np.logspace(-4, -1, 10),
#             'regressor__l1_ratio': [0.01, .1, .5, .9, .98, 1],
#             'scaler': [preprocess]}
#              ]
# grid = GridSearchCV(pipe, multiple_param_grid,n_jobs=-1,return_train_score=True,cv=10)
# grid.fit(X_train, y_train)

In [ ]:
# RIDGE
ridge_param_grid = {'imputer': [KNNImputer()],
            'regressor': [Ridge()],
            'regressor__alpha': np.logspace(-3, 3, 13),
            'scaler': [preprocess]}
ridge_grid = GridSearchCV(pipe, ridge_param_grid,n_jobs=-1,return_train_score=True,cv=10)
ridge_grid.fit(X_train, y_train)

In [ ]:
plot_alpha_score(ridge_grid.cv_results_)

In [ ]:
# LASSO
lasso_param_grid =  {'imputer': [KNNImputer()],
             'regressor': [Lasso()],
            'regressor__alpha': np.logspace(-3, 0, 13),
            'scaler': [preprocess]}
lasso_grid = GridSearchCV(pipe, lasso_param_grid,n_jobs=-1,return_train_score=True,cv=10)
lasso_grid.fit(X_train, y_train)

In [ ]:
plot_alpha_score(lasso_grid.cv_results_)

In [ ]:
# ELASTIC NET
elastic_param_grid = {'imputer': [KNNImputer()],
            'regressor': [ElasticNet()],
            'regressor__alpha': np.logspace(-4, -1, 10),
            'regressor__l1_ratio': [0.01, .1, .5, .9, .98, 1],
            'scaler': [preprocess]}
elastic_grid = GridSearchCV(pipe, elastic_param_grid,n_jobs=-1,return_train_score=True,cv=10)
elastic_grid.fit(X_train, y_train)

In [ ]:
plot_alpha_l1_score(elastic_grid.cv_results_)

### Question 1.6
Visualize the coefficients of the resulting models. Do they agree on which features are
important?

In [ ]:
ridge = ridge_grid.best_estimator_.named_steps['regressor']
lasso = lasso_grid.best_estimator_.named_steps['regressor']
elastic = elastic_grid.best_estimator_.named_steps['regressor']
fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(1,1,1)
ax.set_xlabel("Features")
ax.set_ylabel("Weight")
ax.plot(ridge.coef_, label="Ridge",alpha=0.5)
ax.plot(lasso.coef_, label="Lasso",alpha=0.5)
ax.plot(elastic.coef_, label="ElasticNet",alpha=0.5)
ax.legend()
plt.show()

As seen in plot, there are number of features that got significantly different weights on regressors.
We can try to analize what these features are

In [ ]:
#Let's take a closer look at 10 features with maximal standard deviation of weights among 3 regressors
df= pd.DataFrame()
df['Ridge']=ridge.coef_
df['Lasso']=lasso.coef_
df['ElasticNet']=elastic.coef_
df['std'] = df.std(axis=1)
df['Feature'] = X_train.columns
df.sort_values('std',inplace=True,ascending=False)
df

In [ ]:
most_divergent_features = df.loc[:10,['Ridge','Lasso','ElasticNet','Feature']]
fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot()
ax.set_title("10 most divergent features by weight")
ax.set_xlabel("Feature")
ax.set_ylabel("Weigth")
ax.legend(handles=df.loc[:10,'Feature'])
most_divergent_features.plot(ax=ax,kind="bar",legend=True)

## Task 2 Classification on the Telco-churn dataset (40 points)

You can download the dataset and see it’s description at <br>
https://www.kaggle.com/blastchar/telco-customer-churn

### Question 2.1

Load the data from the file into a DataFrame

In [ ]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

df.head()
# df.iloc["2229-VWQJH"]

### Question 2.2

Split data into training and test set. Build a pipeline for dealing with categorical variables.
Evaluate Logistic Regression and nearest centroids using
cross-validation. How different are the results? How does scaling the continuous features with
StandardScaler influence the results?

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

y = df.Churn
x = df.drop(["Churn", "customerID"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(pd.get_dummies(x), y, random_state=0)

knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(X_train, y_train)
print(f"accuracy: {knn.score(X_test, y_test):.2f}")

### Question 2.3

Tune the parameters using GridSearchCV. Do the results improve?

In [ ]:
# your code here

### Question 2.4

Change the cross-validation strategy from ‘stratified k-fold’ to ‘kfold’ with shuffling. Do the
parameters that are found change? Do they change if you change the random seed of the
shuffling? Or if you change the random state of the split into training and test data?

In [ ]:
# your code here

Good Luck!